In [1]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.remote.webelement import WebElement

In [44]:
options = webdriver.FirefoxProfile("/home/mkypr/.mozilla/firefox/o8g3xqmv.Bot")
# options.add_argument('--user-data-dir=/home/mkypr/.config/google-chrome/Default')
# driver = webdriver.Chrome(chrome_options=options)

browser = webdriver.Firefox(firefox_profile=options)
browser.set_window_position(0, 0, windowHandle="current")
browser.get("https://www.google.com/search?q=ebay+kleinanzeigen")
baseurl = "https://www.ebay-kleinanzeigen.de"
browser.get(baseurl + "/")
searchxpath = "/html/body/header/section[2]/div/div/div[1]/div/form/div/div[1]/div/div[1]/div/input"
findxpath = "/html/body/header/section[2]/div/div/div[1]/div/form/div/div[3]/button"
resultxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[4]/div/ul/*[article]"
nextxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[6]/div[2]/div/div[2]/a"
nextxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[6]/div[2]/div/div[2]/a"
nextxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[6]/div[2]/div/div[last()]/a"
pricexpath = "/html/body/div[1]/div[2]/div/div[3]/div[1]/div[1]/form/div/section[3]/div/div/fieldset/input[2]"
priceconfirmxpath = "/html/body/div[1]/div[2]/div/div[3]/div[1]/div[1]/form/div/section[3]/div/div/button"
# nextxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[6]/div[2]/div/div[2]/a[1]"
# nextxpath = "/html/body/div[1]/div[2]/div/div[3]/div[2]/div[6]/div[2]/div/div[3]/a"
onlyoffersxpath = "/html/body/div[1]/div[2]/div/div[3]/div[1]/div[1]/form/div/section[4]/div/ul/li[1]/a"
bannerxpath  = "/html/body/div[2]/div/div/div/div/div[3]"
coookiexpath = "//*[@id=\"gdpr-banner-accept\"]"

def getelems(xpath, timeout=5):
    cookiebtn = None
    try:
        tmp = WebDriverWait(browser, timeout).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath)))
        cookiebtn = tmp
    except TimeoutException:
        print("timeout")
    except StaleElementReferenceException:
        print("stale")
    return cookiebtn

def getelem(xpath, timeout=10):
    cookiebtn = None
    try:
        cookiebtn = WebDriverWait(browser, timeout).until(
            EC.element_to_be_clickable((By.XPATH, xpath)))
    except TimeoutException:
        print("timeout")
    except StaleElementReferenceException:
        print("stale")
    return cookiebtn

def doclick(elem):
    if not elem:
        return
    action = webdriver.common.action_chains.ActionChains(browser)
    action.move_to_element_with_offset(elem, 10, 10)
    action.click()
    action.perform()

nsites = 5
nmessage = 10
query = "playstation"
queries = [query[:i] + query[i + 1:] for i in range(1, len(query))]
offers = []
goodresults = []
for query in queries:
    print("running query:", query)
    getelem(searchxpath).send_keys(query)
    doclick(getelem(findxpath))
    onlyoffers = getelem(onlyoffersxpath)
    if "angebot" in onlyoffers.text.lower():
        doclick(onlyoffers)
    priceto = getelem(pricexpath)
    doclick(priceto)
    priceto.send_keys("100")
    doclick(getelem(priceconfirmxpath))
    i = nsites
    while True:
        goodresults.clear()
        results = getelems(resultxpath)
        # print(len(results))
        for result in results: 
            datahref = ""
            result: WebElement
            try: 
                article = result.find_element(By.XPATH, ".//article") 
                datahref = article.get_attribute("data-href")
            except Exception as e: 
                print(e)
                continue
            # dttt = article.find_element(By.XPATH, ".//div[2]/div[1]/div[2]")
            goodresults.append(baseurl + datahref)
            offers.append({"href": datahref, "text": result.text})
        if i == nsites:
            for link in goodresults[4:nmessage+4]:
                browser.get(link)
                optx = getelem("/html/body/div[2]/div/div[1]/div/a", 5)
                if optx: 
                    doclick(optx)
                doclick(getelem("/html/body/div[1]/div[2]/div/section[1]/section/aside/div[1]/div/div[1]/ul/li[1]/button"))
                textarea = getelem("/html/body/div[2]/div/div/form/section/fieldset/div[1]/div/textarea")
                doclick(textarea)
                textarea.send_keys("Hallo")
                input()
                doclick(getelem("/html/body/div[2]/div/div/form/section/fieldset/div[5]/button", 3))
                browser.back()

        print(len(offers))  
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)");
        nextlink = getelem(nextxpath)
        if not nextlink: 
            break
        doclick(nextlink)
        i -= 1
        if i == 0: 
            break
browser.close()
import json
open("offers.json", 'w').write(json.dumps(offers))

<ipython-input-44-36879f5c303c>:1: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  options = webdriver.FirefoxProfile("/home/mkypr/.mozilla/firefox/o8g3xqmv.Bot")
<ipython-input-44-36879f5c303c>:5: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  browser = webdriver.Firefox(firefox_profile=options)


running query: paystation
Message: The element reference of <li class="ad-listitem lazyload-item   "> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.jsm:464:5
element.resolveElement@chrome://remote/content/marionette/element.js:681:11
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:254:26
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:262:29
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:262:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:79:29

Message: The element reference of <li class="ad-listitem lazyload-item   "> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been re

WebDriverException: Message: Failed to decode response from marionette


In [37]:
query = "playstation"
queries = [query[:i] + query[i+1:] for i in range(1, len(query))]
print(queries)

['paystation', 'plystation', 'plastation', 'playtation', 'playsation', 'playsttion', 'playstaion', 'playstaton', 'playstatin', 'playstatio']
